### Quick Refresher on Matrix Math, Rotations and Transformations 
 

In [2]:
import numpy as np 
import numpy.typing as npt

If **A** is an \( m by n \) matrix and **B** is an \( n by p \) matrix,

$$
\mathbf{A} = \begin{pmatrix}
a_{11} & a_{12} & \cdots & a_{1n} \\
a_{21} & a_{22} & \cdots & a_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
a_{m1} & a_{m2} & \cdots & a_{mn}
\end{pmatrix}, \quad
\mathbf{B} = \begin{pmatrix}
b_{11} & b_{12} & \cdots & b_{1p} \\
b_{21} & b_{22} & \cdots & b_{2p} \\
\vdots & \vdots & \ddots & \vdots \\
b_{n1} & b_{n2} & \cdots & b_{np}
\end{pmatrix}
$$

the **matrix product** \( C =AB \)  is defined to be the \( m by p \) matrix

$$
\mathbf{C} = \begin{pmatrix}
c_{11} & c_{12} & \cdots & c_{1p} \\
c_{21} & c_{22} & \cdots & c_{2p} \\
\vdots & \vdots & \ddots & \vdots \\
c_{m1} & c_{m2} & \cdots & c_{mp}
\end{pmatrix}
\quad \text{such that} \quad
c_{ij} = a_{i1}b_{1j} + a_{i2}b_{2j} + \cdots + a_{in}b_{nj}
= \sum_{k=1}^{n} a_{ik} b_{kj}
$$

for \( i = 1, \dots, m \) and \( j = 1, \dots, p \).





Therefore, **AB** can also be written as

$$
\mathbf{C} =
\begin{pmatrix}
a_{11}b_{11} + \cdots + a_{1n}b_{n1} & a_{11}b_{12} + \cdots + a_{1n}b_{n2} & \cdots & a_{11}b_{1p} + \cdots + a_{1n}b_{np} \\
a_{21}b_{11} + \cdots + a_{2n}b_{n1} & a_{21}b_{12} + \cdots + a_{2n}b_{n2} & \cdots & a_{21}b_{1p} + \cdots + a_{2n}b_{np} \\
\vdots & \vdots & \ddots & \vdots \\
a_{m1}b_{11} + \cdots + a_{mn}b_{n1} & a_{m1}b_{12} + \cdots + a_{mn}b_{n2} & \cdots & a_{m1}b_{1p} + \cdots + a_{mn}b_{np}
\end{pmatrix}
$$

Thus the product AB is defined if and only if the number of columns in A equals the number of rows in B,in this case n.

In [47]:
# Matrix Multiplication
def dot_1d(arr1:npt.NDArray, arr2:npt.NDArray): 
    assert arr1.shape == arr2.shape
    assert arr1.ndim == 1 or len(arr1.shape) == 1
    s = 0
    for i in range(len(arr1)): 
        s += arr1[i]* arr2[i]
    return s

def matmul(matrix_a:npt.NDArray, matrix_b:npt.NDArray):
    assert matrix_a.ndim == 2 and matrix_b.ndim == 2
    mat_a_rows, mat_a_cols = matrix_a.shape
    mat_b_rows, mat_b_cols = matrix_b.shape
    assert mat_a_cols == mat_b_rows
    mat_c = np.zeros(shape=(mat_a_rows, mat_b_cols ))
    for i in range(mat_a_rows): 
        for j in range(mat_b_cols): 
            mat_c[i,j] = dot_1d(matrix_a[i], matrix_b[:,j])
    return mat_c

mat_a = np.array([[1,2],[3,4],[3,4]])
mat_b = np.array([[5,6],[7,8]])
assert np.array_equal(mat_a @ mat_b , matmul(mat_a,mat_b))